In [1]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

2024-01-02 13:57:30.088514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 13:57:30.088543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 13:57:30.089246: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 13:57:30.093391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 13:57:30.750967: W tensorflow/compiler/tf2

In [2]:
def read_csv_files_from_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .csv extension
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [3]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [4]:
results = read_csv_files_from_folder('results/')
results.replace({'ftl_mlp_initializer': 'Fair Transition Loss', 'adversarial_debiasing_initializer': 'Adversarial Debiasing', 'gerry_fair_classifier_initializer': 'Gerry Fair Classifier', 'prejudice_remover_initializer': 'Prejudice Remover', 'simple_mlp_initializer': 'Standard MLP (baseline)', 'adaptative_priority_reweighting_classifier_initializer': 'Adaptative Priority Reweighting'}, inplace=True)
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [5]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_356879/3340472993.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.31593009  0.31000841  0.33131997  0.32331717  0.32900626  0.31235223
  0.28531143  0.31725419  0.31646536  0.34244247  0.3265771   0.30718997
  0.34741723  0.32744447  0.33941611  0.48462391  0.51246768  0.51909213
  0.49597118  0.51770609  0.479619    0.48794997  0.52038962  0.48669612
  0.50684677  0.52402237  0.54340012  0.46845419  0.49362086  0.49952734
  0.48248795  0.42436149  0.45085125  0.3812566   0.38153244  0.40890686
  0.4388842   0.34968623  0.39085102  0.3838675   0.3004171   0.40080999
  0.33641738  0.31660706  0.27532054  0.27532054  0.48505661  0.51688789
  0.48729668  0.50971284  0.46909481  0.48556223  0.53240448  0.46959752
  0.52591821  0.50330756  0.50417042  0.51763861  0.53081121  0.52271068
  0.5265485   0.50103902  0.27542244  0.2385041   0.29544285  0.28585208
  0.28117914  0.30924409  0.25298732  0

In [6]:
display(results)

,Unnamed: 0,dataset,method,fitness_rule,fitness,Accuracy,Mathew Correlation,Equalized Odds,Statistical Parity,Equal Opportunity,Performance,Fairness,Fitness Rule,Fitness Rule Abvr
0,2,Compas Recidivism,Adversarial Debiasing,mcc_parity,0.202105,0.663695,0.315930,0.095539,0.113825,0.059224,0.315930,0.113825,Max(Mathew Correlation - Statistical Parity),Max(MCC - Stat. Parity)
1,7,Compas Recidivism,Adversarial Debiasing,mcc_odds,0.220379,0.633712,0.259502,0.039123,0.024473,0.034721,0.259502,0.039123,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2,12,Compas Recidivism,Adversarial Debiasing,mcc_opportunity,0.285652,0.660454,0.303159,0.010621,0.010864,0.017507,0.303159,0.017507,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
3,17,Compas Recidivism,Adversarial Debiasing,acc_parity,0.586824,0.666937,0.328174,0.049345,0.080113,0.019058,0.666937,0.080113,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
4,22,Compas Recidivism,Adversarial Debiasing,acc_odds,0.611710,0.676661,0.344075,0.064952,0.085725,0.060409,0.676661,0.064952,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,265,Bank Marketing,Adaptative Priority Reweighting,mcc_odds,0.425545,0.898983,0.467394,0.041848,0.035018,0.065836,0.467394,0.041848,Max(Mathew Correlation - Equalized Odds),Max(MCC - Eq. Opp.)
2105,266,Bank Marketing,Adaptative Priority Reweighting,mcc_opportunity,0.458816,0.898819,0.485940,0.015736,0.057399,0.027124,0.485940,0.027124,Max(Mathew Correlation - Equal Opportunity),Max(MCC - Eq. Odds)
2106,267,Bank Marketing,Adaptative Priority Reweighting,acc_parity,0.895573,0.898163,0.439099,0.093523,0.002590,0.183730,0.898163,0.002590,Max(Accuracy - Statistical Parity),Max(Acc - Stat. Parity)
2107,268,Bank Marketing,Adaptative Priority Reweighting,acc_odds,0.830674,0.899475,0.453680,0.068801,0.070182,0.114781,0.899475,0.068801,Max(Accuracy - Equalized Odds),Max(Acc - Eq. Opp.)


In [7]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [8]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [9]:
methods = ['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier',
 'Adaptative Priority Reweighting']
methods

['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier',
 'Adaptative Priority Reweighting']

In [10]:
if os.path.exists('multi_aso_data_list.json'):
    with open('multi_aso_data_list.json') as file:
        multi_aso_data_list = json.load(file)
else:    
    multi_aso_data_list = []
    for d in datasets:
        multi_aso_data = []
        for f in fitness_rules:
            methods_results = []
            for m in methods:
                r = results.loc[ (results['dataset'] == d) &
                                     (results['fitness_rule'] == f) &
                                     (results['method'] == m) ]\
                            .fitness.tolist()
                if len(r) == 0:
                    r = [-1]
                methods_results.append(r)
            min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
            multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
    with open('multi_aso_data_list.json', 'w') as file:
        json.dump(multi_aso_data_list, file)

Model comparisons: 100%|█████████▉| 14985/15000 [00:33<00:00, 451.92it/s]


In [11]:
aso_df_resume = []
reverse_aso_df_resume = []
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['method'] = methods
    aso_df['dataset'] = dataset
    aso_df['fitness_rule'] = fitness_rule
    aso_df = aso_df[aso_df['method'] == 'Fair Transition Loss' ]
    aso_df = aso_df.drop(['Fair Transition Loss'], axis=1)
    aso_df = aso_df.drop(['method'], axis=1)
    aso_df_resume.append(aso_df)

    reverse_aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods).transpose()
    mapping = dict()
    for i, m in enumerate(methods):
        mapping[reverse_aso_df.columns[i]] = m + ' (reverse)'
    reverse_aso_df = reverse_aso_df.rename(columns=mapping)
    reverse_aso_df['method'] = methods
    reverse_aso_df['dataset'] = dataset
    reverse_aso_df['fitness_rule'] = fitness_rule
    reverse_aso_df = reverse_aso_df[reverse_aso_df['method'] == 'Fair Transition Loss' ]
    reverse_aso_df = reverse_aso_df.drop(['Fair Transition Loss (reverse)'], axis=1)
    reverse_aso_df = reverse_aso_df.drop(['method'], axis=1)
    reverse_aso_df_resume.append(reverse_aso_df)

print('Significance Testing')
significance = pd.concat(aso_df_resume)
significance = significance.set_index(['fitness_rule', 'dataset'])
significance = significance.sort_values(by=['fitness_rule', 'dataset'])
#display(pd.transpose(significance))
formatted_significance = significance.applymap(lambda x: '\\textbf{' + f'{x:.2f}' + '}' if x < 0.5 else f'{x:.2f}' )
formatted_significance.to_latex('tables/significance_resume.tex')

reverse_significance = pd.concat(reverse_aso_df_resume)
reverse_significance = reverse_significance.set_index(['fitness_rule', 'dataset'])
reverse_significance = reverse_significance.sort_values(by=['fitness_rule', 'dataset'])
reverse_formatted_significance = reverse_significance.applymap(lambda x: '\\textit{' + f'{x:.2f}' + '}' if x < 0.5 else f'{x:.2f}' )
reverse_formatted_significance.to_latex('tables/reverse_significance_resume.tex')

/tmp/ipykernel_356879/3353810814.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  formatted_significance = significance.applymap(lambda x: '\\textbf{' + f'{x:.2f}' + '}' if x < 0.5 else f'{x:.2f}' )


Significance Testing


/tmp/ipykernel_356879/3353810814.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  reverse_formatted_significance = reverse_significance.applymap(lambda x: '\\textit{' + f'{x:.2f}' + '}' if x < 0.5 else f'{x:.2f}' )


In [12]:
for col, rev_col in zip(formatted_significance.columns, reverse_formatted_significance.columns):
    formatted_significance[col] = formatted_significance[col] + (' (' + reverse_formatted_significance[rev_col] + ')')
formatted_significance.to_latex('tables/combined_significance_resume.tex')
formatted_significance

Standard MLP (baseline)  \
fitness_rule    dataset                                     
acc_odds        Adult Income         \textbf{0.01} (1.00)   
                Bank Marketing                0.77 (1.00)   
                Compas Recidivism    \textbf{0.01} (1.00)   
                German Credit        1.00 (\textit{0.36})   
acc_opportunity Adult Income                  0.90 (1.00)   
                Bank Marketing                1.00 (0.76)   
                Compas Recidivism    \textbf{0.01} (0.99)   
                German Credit                 1.00 (0.60)   
acc_parity      Adult Income         \textbf{0.01} (1.00)   
                Bank Marketing       \textbf{0.24} (1.00)   
                Compas Recidivism    \textbf{0.00} (1.00)   
                German Credit                 1.00 (0.73)   
mcc_odds        Adult Income         \textbf{0.04} (1.00)   
                Bank Marketing       \textbf{0.44} (1.00)   
                Compas Recidivism    \textbf{0.01} (1.00)   
                German Credit                 1.00 (1.00)   
mcc_opportunity Adult Income         \textbf{0.01} (1.00)   
                Bank Marketing                0.81 (1.00)   
                Compas Recidivism    \textbf{0.00} (1.00)   
                German Credit                 1.00 (0.84)   
mcc_parity      Adult Income         \textbf{0.00} (1.00)   
                Bank Marketing       \textbf{0.01} (1.00)   
                Compas Recidivism    \textbf{0.01} (1.00)   
                German Credit        \textbf{0.28} (1.00)   

                                  Adversarial Debiasing     Prejudice Remover  \
fitness_rule    dataset                                                         
acc_odds        Adult Income       \textbf{0.21} (1.00)  \textbf{0.19} (1.00)   
                Bank Marketing     \textbf{0.39} (1.00)           0.80 (1.00)   
                Compas Recidivism  \textbf{0.45} (1.00)  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.11} (1.00)  1.00 (\textit{0.16})   
acc_opportunity Adult Income                0.96 (1.00)           1.00 (0.50)   
                Bank Marketing     \textbf{0.38} (1.00)           0.81 (1.00)   
                Compas Recidivism           0.80 (1.00)  \textbf{0.00} (1.00)   
                German Credit               0.63 (1.00)           1.00 (0.81)   
acc_parity      Adult Income       \textbf{0.28} (1.00)  \textbf{0.32} (1.00)   
                Bank Marketing              1.00 (0.56)           1.00 (0.73)   
                Compas Recidivism  1.00 (\textit{0.48})  \textbf{0.11} (1.00)   
                German Credit      \textbf{0.25} (1.00)  1.00 (\textit{0.40})   
mcc_odds        Adult Income       \textbf{0.25} (1.00)  \textbf{0.44} (1.00)   
                Bank Marketing     \textbf{0.18} (1.00)  \textbf{0.12} (1.00)   
                Compas Recidivism           0.56 (1.00)  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.13} (1.00)           1.00 (0.89)   
mcc_opportunity Adult Income       \textbf{0.00} (1.00)  \textbf{0.04} (1.00)   
                Bank Marketing     \textbf{0.18} (1.00)  \textbf{0.24} (1.00)   
                Compas Recidivism  1.00 (\textit{0.16})  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.21} (1.00)           0.85 (1.00)   
mcc_parity      Adult Income       \textbf{0.15} (1.00)           1.00 (1.00)   
                Bank Marketing     \textbf{0.00} (1.00)  \textbf{0.00} (1.00)   
                Compas Recidivism  \textbf{0.24} (1.00)  \textbf{0.00} (1.00)   
                German Credit      \textbf{0.31} (1.00)  \textbf{0.38} (1.00)   

                                  Gerry Fair Classifier  \
fitness_rule    dataset                                   
acc_odds        Adult Income       \textbf{0.00} (1.00)   
                Bank Marketing              1.00 (0.80)   
                Compas Recidivism  \textbf{0.01} (1.00)   
                German Credit      1.00 (\tex

In [13]:
grouped_results = results\
    .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
grouped_results

fitness  \
                                                                           mean   
Fitness Rule Abvr       dataset       method                                      
Max(Acc - Eq. Odds)     Adult Income  Adaptative Priority Reweighting  0.828577   
                                      Prejudice Remover                0.809534   
                                      Fair Transition Loss             0.787013   
                                      Adversarial Debiasing            0.756054   
                                      Standard MLP (baseline)          0.752260   
...                                                                         ...   
Max(MCC - Stat. Parity) German Credit Prejudice Remover                0.233727   
                                      Standard MLP (baseline)          0.223085   
                                      Gerry Fair Classifier            0.220550   
                                      Adaptative Priority Reweighting  0.216517   
                                      Adversarial Debiasing            0.200070   

                                                                                 \
                                                                            std   
Fitness Rule Abvr       dataset       method                                      
Max(Acc - Eq. Odds)     Adult Income  Adaptative Priority Reweighting  0.012771   
                                      Prejudice Remover                0.022701   
                                      Fair Transition Loss             0.083934   
                                      Adversarial Debiasing            0.028345   
                                      Standard MLP (baseline)          0.041790   
...                                                                         ...   
Max(MCC - Stat. Parity) German Credit Prejudice Remover                0.087339   
                                      Standard MLP (baseline)          0.095145   
                                      Gerry Fair Classifier            0.088103   
                                      Adaptative Priority Reweighting  0.094516   
                                      Adversarial Debiasing            0.172675   

                                                                      Performance  \
                                                                             mean   
Fitness Rule Abvr       dataset       method                                        
Max(Acc - Eq. Odds)     Adult Income  Adaptative Priority Reweighting    0.846582   
                                      Prejudice Remover                  0.845788   
                                      Fair Transition Loss               0.825826   
                                      Adversarial Debiasing              0.847730   
                                      Standard MLP (baseline)            0.849309   
...                                                                           ...   
Max(MCC - Stat. Parity) German Credit Prejudice Remover                  0.328818   
                                      Standard MLP (baseline)            0.329618   
                                      Gerry Fair Classifier              0.291429   
                                      Adaptative Priority Reweighting    0.321270   
                                      Adversarial Debiasing              0.367673   

                                                                                 \
                                                                            std   
Fitness Rule Abvr       dataset       method                                      
Max(Acc - Eq. Odds)     Adult Income  Adaptative Priority Reweighting  0.004174   
                                      Prejudice Remover                0.003691   
                                      Fair Transition Loss             0.069484   
                                      Adversarial Debiasi

In [14]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
grouped_results[selected_columns].to_latex('tables/grouped_results.tex')